## Imports

In [1]:
from mesa import Agent, Model 
from mesa.space import MultiGrid
from mesa.time import SimultaneousActivation
from mesa.datacollection import DataCollector
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128
import numpy as np
import pandas as pd
import random
import time
import datetime

## Crear el modelo

In [17]:
    
class Habitacion(Model):
    
    def __init__(self, m, n, num_agentes, por_celdas_sucias):
        self.num_agentes = num_agentes
        self.por_celdas_sucias = por_celdas_sucias
        self.por_celdas_limpias = 1 - por_celdas_sucias
        self.grid = MultiGrid(m, n, True)
        self.schedule = SimultaneousActivation(self)
               
        # se colocan celdas sucias de forma aleatoria
        celdas_sucias = int((m * n) * por_celdas_sucias)
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in range(celdas_sucias):
            celda_vacia = random.choice(lista_celdas_vacias)
            piso = Piso(celda_vacia, self)
            piso.estado = piso.SUCIO
            self.grid.place_agent(piso, celda_vacia)
            self.schedule.add(piso)
            lista_celdas_vacias.remove(celda_vacia)
        
        # Posicionar celdas limpias
        lista_celdas_vacias = list(self.grid.empties)
        for celdas in lista_celdas_vacias:
            piso = Piso(celdas, self)
            self.grid.place_agent(piso, celdas)
            self.schedule.add(piso)
        
        # Posicionar agentes aspiradoras
        for i in range(num_agentes):
            aspiradora = Aspiradora(i, self)
            self.grid.place_agent(aspiradora, (1,1))
            self.schedule.add(aspiradora)
            
        self.colectordatos = DataCollector(
            model_reporters={'Habitacion': crear_espacio},
            agent_reporters={'Movimientos': lambda a: getattr(a, 'movimientos', None)}
        )
    
    def step(self):
        self.colectordatos.collect(self)
        self.schedule.step()    
    
    def todasceldaslimpias(self):
        celdas_limpias = 0
        for celda in self.grid.coord_iter():
            contenido_celda, x, y = celda
            for contenido in contenido_celda:
                if isinstance(contenido, Piso) and contenido.estado == contenido.LIMPIO:
                    celdas_limpias = celdas_limpias + 1
        
        self.por_celdas_limpias = celdas_limpias / (self.grid.width * self.grid.height)
        if self.por_celdas_limpias == 1:
            return True
        else:
            return False



class Aspiradora(Agent):
    
    def __init__(self, id_unico, modelo):
        super().__init__(id_unico, modelo)
        self.movimientos = 0
        self.nxt_pos = None
        
        
    def step(self):
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=True,
            include_center=True)
        
        # se define el siguiente estado que va a tener el piso para la siguiente iteracion
        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos:
                vecino.siguiente_estado = vecino.estado
                if vecino.siguiente_estado == vecino.SUCIO: 
                    vecino.siguiente_estado = vecino.LIMPIO
                    self.nxt_pos = self.pos
                else:
                    vecindario = self.model.grid.get_neighborhood(
                        self.pos,
                        moore=True,
                        include_center=False)
                    nxt_pos = self.random.choice(vecindario)
                    self.nxt_pos = nxt_pos
                break
        
    def advance(self):
        
        # se actualiza el estado del piso
        vecinos = self.model.grid.get_neighbors(
            self.pos,
            moore=False,
            include_center=True)
        
        for vecino in vecinos:
            if isinstance(vecino, Piso) and vecino.pos == self.pos: 
                vecino.estado = vecino.siguiente_estado
                break
        
        if self.pos != self.nxt_pos:
            self.movimientos = self.movimientos + 1
            
        self.model.grid.move_agent(self, self.nxt_pos)

            
class Piso(Agent):
    
    SUCIO = 50
    LIMPIO = 100
    
    def __init__(self, pos, modelo, estado=LIMPIO):
        super().__init__(pos, modelo)
        self.x, self.y = pos
        self.estado = estado
        self.siguiente_estado = None
        

def crear_espacio(modelo):
    habitacion = np.zeros((modelo.grid.width, modelo.grid.height))
    for celda in modelo.grid.coord_iter():
        contenido_celda, x, y = celda
        for contenido in contenido_celda:
            if isinstance(contenido, Aspiradora):
                habitacion[x][y] = 2
            else:
                habitacion[x][y] = contenido.estado
    return habitacion

Correr el modelo

In [61]:
# Datos de la habitacion:Dimensiones
M = 15
N = 15

# Numero de agentes
NUM_AGENTES = 8

# Porcentaje de celdas inicialmente sucias:
PORCENTAJE_CELDAS_SUCIAS = 0.7

# Tiempo máximo de ejecución (segundos)
TIEMPO_MAXIMO_EJECUCION =  1

start_time = time.time()
tiempo_inicio = str(datetime.timedelta(seconds=TIEMPO_MAXIMO_EJECUCION))
modelo = Habitacion(M, N, NUM_AGENTES, PORCENTAJE_CELDAS_SUCIAS)

while((time.time() - start_time) < TIEMPO_MAXIMO_EJECUCION and not modelo.todasceldaslimpias()):
    modelo.step()
    
# Imprimimos el tiempo que le tomó correr al modelo.
tiempo_ejecucion = str(datetime.timedelta(seconds=(time.time() - start_time)))

## Visualización

In [62]:
todas_habitaciones = modelo.colectordatos.get_model_vars_dataframe()

In [63]:
%%capture

fig, axs = plt.subplots(figsize=(7,7))
axs.set_xticks([])
axs.set_yticks([])
patch = plt.imshow(todas_habitaciones.iloc[0][0], cmap='nipy_spectral')

def animate(i):
    patch.set_data(todas_habitaciones.iloc[i][0])
    
anim = animation.FuncAnimation(fig, animate, frames=len(todas_habitaciones))

## Informe

In [49]:
anim

In [64]:
movimientos = modelo.colectordatos.get_agent_vars_dataframe()

print('Tiempo total de limpieza:', tiempo_ejecucion)
print('Porcentaje de celdas limpiadas:', modelo.por_celdas_limpias * 100)
print('Número de movimientos realizados en total:', movimientos.tail()['Movimientos'].sum())

Tiempo total de limpieza: 0:00:00.297206
Porcentaje de celdas limpiadas: 100.0
Número de movimientos realizados en total: 1097.0
